In [3]:
import pandas as pd
import numpy as np
import re
import pickle
import time
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import permutations
import json, os
from collections import OrderedDict
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import lightgbm as lgb

from matplotlib import pyplot

In [4]:
pd.set_option('display.max_columns', None)

df = pd.read_csv('../../BPIC_Classification/processed_dataset/P_BPIC12_agg.csv')
df.head()

,A_SUBMITTED-COMPLETE,A_PARTLYSUBMITTED-COMPLETE,A_PREACCEPTED-COMPLETE,W_Completeren aanvraag-SCHEDULE,W_Completeren aanvraag-START,A_ACCEPTED-COMPLETE,O_SELECTED-COMPLETE,A_FINALIZED-COMPLETE,O_CREATED-COMPLETE,O_SENT-COMPLETE,W_Nabellen offertes-SCHEDULE,W_Completeren aanvraag-COMPLETE,W_Nabellen offertes-START,W_Nabellen offertes-COMPLETE,O_SENT_BACK-COMPLETE,W_Valideren aanvraag-SCHEDULE,W_Valideren aanvraag-START,A_REGISTERED-COMPLETE,A_APPROVED-COMPLETE,O_ACCEPTED-COMPLETE,A_ACTIVATED-COMPLETE,W_Valideren aanvraag-COMPLETE,O_CANCELLED-COMPLETE,W_Wijzigen contractgegevens-SCHEDULE,A_DECLINED-COMPLETE,A_CANCELLED-COMPLETE,W_Afhandelen leads-SCHEDULE,W_Afhandelen leads-START,W_Afhandelen leads-COMPLETE,O_DECLINED-COMPLETE,W_Nabellen incomplete dossiers-SCHEDULE,W_Nabellen incomplete dossiers-START,W_Nabellen incomplete dossiers-COMPLETE,W_Beoordelen fraude-SCHEDULE,W_Beoordelen fraude-START,W_Beoordelen fraude-COMPLETE,112.0,Unnamed: 37,10862.0,10913.0,11049.0,10629.0,11120.0,10809.0,10912.0,11201.0,11119.0,10861.0,11203.0,11181.0,11189.0,10609.0,11111.0,10982.0,11019.0,11180.0,10899.0,10138.0,11002.0,11122.0,10889.0,10972.0,11121.0,10939.0,11029.0,11009.0,11000.0,10863.0,11169.0,11179.0,11001.0,10971.0,10228.0,11202.0,10789.0,10881.0,10909.0,10188.0,10910.0,10929.0,10931.0,11259.0,11200.0,10779.0,10880.0,10914.0,10859.0,11339.0,10933.0,11079.0,10932.0,10935.0,11254.0,11003.0,10125.0,11269.0,10821.0,11289.0,10124.0,11299.0,11309.0,11300.0,11302.0,11319.0,11304.0,Complete Timestamp,label,Variant,AMOUNT,A_SUBMITTED,A_PARTLYSUBMITTED,A_PREACCEPTED,W_Completeren aanvraag,A_ACCEPTED,O_SELECTED,A_FINALIZED,O_CREATED,O_SENT,W_Nabellen offertes,O_SENT_BACK,W_Valideren aanvraag,A_REGISTERED,A_APPROVED,O_ACCEPTED,A_ACTIVATED,O_CANCELLED,W_Wijzigen contractgegevens,A_DECLINED,A_CANCELLED,W_Afhandelen leads,O_DECLINED,W_Nabellen incomplete dossiers,W_Beoordelen fraude,COMPLETE,SCHEDULE,START
0,1,1,1,1,1,1,1,1,1,1,1,1,3,3,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,5,5,2,4,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2011-10-06 09:44:45.718269184,1,Variant 613,20000,1,1,1,3,1,1,1,1,1,7,1,3,1,1,1,1,0,0,0,0,0,0,0,0,18,3,5
1,1,1,1,1,2,1,2,1,2,2,2,2,4,4,1,1,3,1,1,1,1,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4,12,5,0,4,0,4,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2011-10-05 13:15:29.379769344,1,Variant 614,5000,1,1,1,5,1,2,1,2,2,10,1,7,1,1,1,1,1,0,0,0,0,0,0,0,26,4,9
2,1,1,1,1,4,1,3,1,3,3,3,4,11,11,1,1,1,1,1,1,1,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,4,6,0,0,4,0,0,0,5,17,4,7,2,2,2,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2011-10-18 03:47:58.488067584,0,Variant 615,7000,1,1,1,9,1,3,1,3,3,25,1,3,1,1,1,1,2,1,0,0,0,0,0,0,37,6,16
3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2011-10-01 15:11:21.440333312,1,Variant 1,15000,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,0,0
4,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2011-10-01 15:15:53.677666560,1,Variant 1,5000,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,0,0


In [11]:
class AccuracyScore:

    def __init__(self, X_train, y_train, X_test, y_test):
        self.self = self
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test

    def preprocessing(self):
        self['Complete Timestamp'] = pd.to_datetime(self['Complete Timestamp'], format='%Y/%m/%d %H:%M:%S.%f')
        self['Complete Timestamp'] = (self['Complete Timestamp'] - self['Complete Timestamp'].min()) / np.timedelta64(1,'M')
        
        for i in self['Variant']:
            self['Variant'] = self['Variant'].str.replace('Variant', '')

        self['Variant'] = self['Variant'].astype(int)

        return df
    
    def data_split(self):
        X = df.drop('label', axis=1)
        y = df['label']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2023, shuffle=True)
        
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        
        return X_train, y_train, X_test, y_test        
    
    def imbalance(self, X_train, y_train):
        sm = SMOTE(random_state=1)
        X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
        return X_train_res, y_train_res
    
    def DT(self, X_train, y_train, X_test, y_test):
        clf_dt = tree.DecisionTreeClassifier(max_depth=2, random_state=2023)
        clf_dt = clf_dt.fit(X_train, y_train)
        predictions = clf_dt.predict(X_test)
        accuracy = clf_dt.score(X_test, y_test)
        print('DT Accuracy:', accuracy)
        print('DT AUC Score {}'.format(roc_auc_score(y_test, predictions)))

    def RF(self, X_train, y_train, X_test, y_test):
        clf_rf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=2023)
        clf_rf = clf_rf.fit(X_train, y_train)
        predictions = clf_rf.predict(X_test)
        accuracy = clf_rf.score(X_test, y_test)
        print('RF Accuracy:', accuracy)
        print('RF AUC Score {}'.format(roc_auc_score(y_test, predictions)))
    
    def XGB(self, X_train, y_train, X_test, y_test):
        xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=2023)
        xgb.fit(X_train, y_train)
        predictions = xgb.predict(X_test)
        accuracy = xgb.score(X_test, y_test)
        print('XGB Accuracy:', accuracy)
        print('XGB AUC Score {}'.format(roc_auc_score(y_test, predictions)))
    
    def LGBM(self, X_train, y_train, X_test, y_test):
        lgbm = LGBMClassifier(random_state=2023)
        lgbm = lgbm.fit(X_train, y_train)
        predictions = lgbm.predict(X_test)
        accuracy = lgbm.score(X_test, y_test)
        print('LGBM Accuracy:', accuracy)
        print('LGBM AUC Score {}'.format(roc_auc_score(y_test, predictions)))

In [6]:
df = AccuracyScore.preprocessing(df)
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_train, y_train, X_test, y_test = AccuracyScore.data_split(df)
X_train, y_train = AccuracyScore.imbalance(df, X_train, y_train)

In [12]:
AccuracyScore.DT(df, X_train, y_train, X_test, y_test)
AccuracyScore.RF(df, X_train, y_train, X_test, y_test)
AccuracyScore.XGB(df, X_train, y_train, X_test, y_test)
AccuracyScore.LGBM(df, X_train, y_train, X_test, y_test)

DT Accuracy: 0.8838808250572956
DT AUC Score 0.4420529801324503
RF Accuracy: 0.8186911128087598
RF AUC Score 0.9093224656138563
XGB Accuracy: 0.9997453526865292
XGB AUC Score 0.5
LGBM Accuracy: 0.9997453526865292
LGBM AUC Score 0.5


In [8]:
dt_model = DecisionTreeClassifier(max_depth=2, random_state=2023)
dt_model.fit(X_train, y_train)
prediction = dt_model.predict(X_test)
print('AUC Score {}'.format(roc_auc_score(y_test, prediction)))

AUC Score 0.4420529801324503
